In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🏗️ Investment Decisions and Tobin's q Theory

Tobin's q theory provides a simple and intuitive framework for understanding firm investment decisions. It relates the decision to invest in new capital to the comparison between the *value* of that capital and its *cost*.

There are two main versions of q:

1.  **Average q:** Compares the *total market value* of the firm (e.g., stock market valuation) to the *replacement cost* of its *total capital stock*.
    $$ \text{Average } q = \frac{\text{Market Value of Firm}}{\text{Replacement Cost of Capital}} $$
    If Average q > 1, the market values the firm's capital at more than its replacement cost, suggesting investment is profitable.

2.  **Marginal q:** Compares the expected *value added* by one more unit of capital to the *cost* of acquiring that unit. This is closely related to the MPK vs. user cost condition.
    $$ \text{Marginal } q \approx \frac{\text{Marginal Product of Capital (MPK)}}{\text{User Cost of Capital (r + \delta)}} $$
    *(Note: A more precise definition involves the present value of future MPKs relative to the purchase and installation cost).*

This simulation focuses on a simplified **Marginal q**.

# 🚦 The Investment Rule based on q

The core idea is straightforward:

* **If $q > 1$:** The value generated by an additional unit of capital (MPK) exceeds its cost ($r+\delta$). Firms have an incentive to **invest** because installing new capital adds more value than it costs.
* **If $q < 1$:** The value generated by an additional unit of capital is less than its cost. Firms have **no incentive to invest** in new capital and may even want to disinvest (let capital depreciate without replacement).
* **If $q = 1$:** The value equals the cost. The firm is likely at or near its optimal capital stock, with no strong incentive to significantly increase or decrease investment based on this condition alone (investment would just cover depreciation).

The simulation below calculates this simplified Marginal q based on MPK (derived from a Cobb-Douglas function) and the user cost, then determines a hypothetical investment response.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def tobin_q_investment_model(
    A=1.0, alpha=0.33, K_current=50.0, L=100, # Production side
    r=0.04, delta=0.06, # Cost side
    investment_sensitivity=10.0 # How strongly investment reacts to q>1
    ):
    """
    Calculates MPK, user cost, marginal q, and simulates an investment response.

    Args:
        A (float): Total Factor Productivity (TFP).
        alpha (float): Capital share (output elasticity of capital).
        K_current (float): Current level of capital stock.
        L (float): Labor input (assumed constant).
        r (float): Real interest rate.
        delta (float): Depreciation rate.
        investment_sensitivity (float): Parameter scaling the investment response to q>1.
    """
    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    r = max(r, 0)
    delta = max(delta, 0)
    A = max(A, 0.1)
    K_current = max(K_current, 0.1)
    L = max(L, 1)
    investment_sensitivity = max(investment_sensitivity, 0)

    # --- Calculations ---
    # User cost of capital
    user_cost = r + delta

    # MPK at the current capital level K_current
    # MPK = alpha * A * K^(alpha - 1) * L^(1-alpha) (assuming L is constant)
    # Or MPK = alpha * Y / K. Let's use the direct formula.
    # We need L for MPK if production function isn't intensive form
    mpk_at_K_current = alpha * A * (K_current**(alpha - 1)) * (L**(1 - alpha))

    # Simplified Marginal Tobin's q
    if user_cost > 1e-9:
        q_marginal = mpk_at_K_current / user_cost
    else:
        q_marginal = np.inf if mpk_at_K_current > 0 else 0 # Or NaN

    # Optimal K* where MPK = user_cost
    optimal_K_star = np.nan
    if user_cost > 0 and alpha * A * (L**(1-alpha)) > 0:
        try:
            # K* = [ (alpha * A * L**(1-alpha)) / user_cost ]**(1 / (1 - alpha))
            base = (alpha * A * (L**(1-alpha))) / user_cost
            exponent = 1.0 / (1.0 - alpha)
            log_K_star = exponent * np.log(base)
            log_K_star = np.clip(log_K_star, -20, 20) # Avoid extreme values
            optimal_K_star = np.exp(log_K_star)
        except (ValueError, OverflowError, RuntimeWarning):
             optimal_K_star = np.nan


    # --- Investment Response Function ---
    # Simple linear response: Investment = sensitivity * max(0, q - 1)
    # Net Investment (above depreciation)
    net_investment = investment_sensitivity * max(0, q_marginal - 1)
    # Gross Investment = Net Investment + Replacement Investment
    replacement_investment = delta * K_current
    gross_investment = net_investment + replacement_investment


    # --- Plotting Investment Response ---
    fig, ax = plt.subplots(figsize=(8, 5))

    # Generate range of q values for plotting the response function
    q_plot_vals = np.linspace(0, max(2.0, q_marginal * 1.2), 100)
    net_invest_plot = investment_sensitivity * np.maximum(0, q_plot_vals - 1)

    ax.plot(q_plot_vals, net_invest_plot, color='green', linewidth=2, label=f'Net Investment = {investment_sensitivity:.1f} * max(0, q-1)')
    ax.scatter(q_marginal, net_investment, color='red', s=100, zorder=5, label=f'Current q = {q_marginal:.2f}')
    ax.axvline(1, color='grey', linestyle='--', linewidth=1.5, label='q = 1 Threshold')
    ax.axhline(0, color='black', linewidth=0.5)

    ax.set_title("Tobin's q and Net Investment Decision")
    ax.set_xlabel("Marginal Tobin's q (MPK / User Cost)")
    ax.set_ylabel("Net Investment (I_net = Gross I - δK)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=min(-0.5, net_investment*1.2)) # Allow slightly negative view

    plt.tight_layout()
    plt.show()

    # --- Display Summary ---
    if q_marginal > 1.0 + 1e-3:
        signal = f"✅ Invest (q = {q_marginal:.2f} > 1)"
        color = "green"
    elif q_marginal < 1.0 - 1e-3:
        signal = f"❌ Don't Invest Net (q = {q_marginal:.2f} < 1)"
        color = "red"
    elif np.isnan(q_marginal):
         signal = "⚠️ Cannot calculate q"
         color = "grey"
    else:
        signal = f"⚖️ Indifferent (q ≈ 1)"
        color = "orange"

    results_md = f"""
    ### 📊 Investment Analysis:

    * **Current Capital (K):** {K_current:.1f}
    * **Marginal Product of Capital (MPK):** {mpk_at_K_current:.4f}
    * **User Cost (r + δ):** {user_cost:.4f} (r={r:.1%}, δ={delta:.1%})
    * **Marginal Tobin's q (MPK/Cost):** {q_marginal:.3f}
    * **Optimal Capital (K\*):** {optimal_K_star:.1f}
    * **Net Investment (I_net):** {net_investment:.2f}
    * **Gross Investment (I = I_net + δK):** {gross_investment:.2f}
    * **Signal:** <font color='{color}'>**{signal}**</font>
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    tobin_q_investment_model,
    A=FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='TFP (A):', style=style, layout=layout),
    alpha=FloatSlider(value=0.33, min=0.1, max=0.8, step=0.01, description='Capital Share (alpha α):', style=style, layout=layout, readout_format='.2f'),
    K_current=FloatSlider(value=50.0, min=1, max=200, step=1, description='Current Capital (K):', style=style, layout=layout),
    L=FloatSlider(value=100.0, min=10, max=1000, step=10, description='Labor (L, constant):', style=style, layout=layout, readout_format='.0f'),
    r=FloatSlider(value=0.04, min=0.0, max=0.15, step=0.005, description='Interest Rate (r):', style=style, layout=layout, readout_format='.1%'),
    delta=FloatSlider(value=0.06, min=0.0, max=0.15, step=0.005, description='Depreciation (delta δ):', style=style, layout=layout, readout_format='.1%'),
    investment_sensitivity=FloatSlider(value=10.0, min=0, max=50, step=1, description='Invest. Sensitivity:', style=style, layout=layout)
);


<>:128: SyntaxWarning: invalid escape sequence '\*'
<>:128: SyntaxWarning: invalid escape sequence '\*'
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_88503/2819685361.py:128: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))


interactive(children=(FloatSlider(value=1.0, description='TFP (A):', layout=Layout(width='95%'), max=3.0, min=…